In [ ]:
import pandas as pd
import numpy as np
import os
from utils import (
    get_visitor_interests,
    get_exhibitor_category_info,
    calculate_match_score
)
from tqdm import tqdm

In [2]:
# Load processed data 
base_path = '../../source/'
visitors_df  = pd.read_csv(os.path.join(base_path, 'processed_visitors_answers.csv'))
exhibitors_df = pd.read_csv(os.path.join(base_path, 'processed_exhibitors_categories.csv'))
# Get original exhibitor names map
exhibitor_names = exhibitors_df[['exhibitorid', 'exhibitorName']].drop_duplicates().set_index('exhibitorid')['exhibitorName'].to_dict()

In [3]:
visitors_df

,visitor_id,visitor_email,visitor_gender,questionId,questionText,answerId,answerText,answerTypeId
0,67b70a9f2d21f543a1096602,emilija+100_L8gA@bss.mk,F,5c8a78336d41a10da4f730fe,Reason for Attending the Event,5c8a78336d41a10da4f73100,To obtain general information,Answer
1,67b70a9f2d21f543a1096602,emilija+100_L8gA@bss.mk,F,5c8a78336d41a10da4f73227,Which of the following best describes your job...,5c8a78336d41a10da4f73244,Media,Answer
2,67b70a9f2d21f543a1096602,emilija+100_L8gA@bss.mk,F,5c8a78336d41a10da4f73253,Please indicate your company's main area of bu...,5c8a78336d41a10da4f73291,Travel Agent,Answer
3,67b70a9f2d21f543a1096602,emilija+100_L8gA@bss.mk,F,5c8a78336d41a10da4f7336d,What role do you play in the purchasing decisi...,5c8a78336d41a10da4f73371,No influence,Answer
4,67ada1ee197e604dd2722d1b,aleksandar.dimkov+mitt1_n5eA@bss.com.mk,M,5c8a78336d41a10da4f730fe,Reason for Attending the Event,5c8a78336d41a10da4f730ff,To source products and services,Answer
...,...,...,...,...,...,...,...,...
356,uud1nluegdqmisnfzpg75iqg,daniela.p+200_IzcX_QOfF@bss.com.mk,M,5c8a78336d41a10da4f7336d,What role do you play in the purchasing decisi...,5c8a78336d41a10da4f73370,Advisory role,Answer
357,mv0j9yovwhgmtif9l1naciko,daniela.p+200_IzcX_CIHU@bss.com.mk,M,5c8a78336d41a10da4f730fe,Reason for Attending the Event,5c8a78336d41a10da4f73100,To obtain general information,Answer
358,mv0j9yovwhgmtif9l1naciko,daniela.p+200_IzcX_CIHU@bss.com.mk,M,5c8a78336d41a10da4f73227,Which of the following best describes your job...,5c8a78336d41a10da4f7323d,Guided tour services,Answer
359,mv0j9yovwhgmtif9l1naciko,daniela.p+200_IzcX_CIHU@bss.com.mk,M,5c8a78336d41a10da4f73253,Please indicate your company's main area of bu...,5c8a78336d41a10da4f73291,Travel Agent,Answer


In [4]:
exhibitors_df

,exhibitorid,exhibitorName,categoryId,categoryName,parentCategory
0,90556,Turkey Travels,52276,1.5 Resort hotel,1. Accomodation providers
1,90556,Turkey Travels,52280,2.1 Inbound tour operator,2. Tour operators
2,90556,Turkey Travels,52281,2.2 Outbound tour operator,2. Tour operators
3,92462,Russian Travel Company,52273,1.2 Apartments / Residential hotel,1. Accomodation providers
4,92462,Russian Travel Company,52283,2.4 Mass market tour operators,2. Tour operators
...,...,...,...,...,...
178,17729,Sunny Travel Journeys,52282,2.3 Receptive / ground operator,2. Tour operators
179,17729,Sunny Travel Journeys,52272,1.1 Hotel / Hotel chain / Inn,1. Accomodation providers
180,17729,Sunny Travel Journeys,52329,10.5 Zoo,10. Museums & parks
181,17729,Sunny Travel Journeys,52366,"17.3 Banking, investments",17. Insurance companies & banking


In [5]:
exhibitor_names

{90556: 'Turkey Travels',
 92462: 'Russian Travel Company',
 92491: 'Indian Travel Company',
 92492: 'Asia Tourism',
 92493: 'SriLanka Adventures',
 92494: 'Exotic Europe Travels',
 92495: 'Luxury Europe Company',
 92496: 'DreamTravel Company ',
 92497: 'Happy Company 2',
 68142: 'Global Tours Voyages',
 27827: 'Exotic Tours Holidays',
 74870: 'Exotic Tours Journeys',
 56923: 'Royal Tours Expeditions',
 80700: 'Exotic Holidays Services',
 46350: 'Prime Travel Journeys',
 55311: 'Sunny Adventures Journeys',
 31331: 'Exotic Adventures Journeys',
 97818: 'Sunny Trips Adventures',
 13033: 'Prime Vacations Expeditions',
 15881: 'Royal Trips Tours',
 10846: 'Royal Adventures Adventures',
 98889: 'Exotic Tours Tours',
 18004: 'Exotic Getaways Journeys',
 44942: 'Royal Holidays Adventures',
 48028: 'Prime Adventures Journeys',
 72153: 'Elite Adventures Holidays',
 32278: 'Exotic Holidays Holidays',
 51839: 'Dream Getaways Expeditions',
 30134: 'Global Holidays Expeditions',
 54579: 'Prime Vaca

In [6]:
exhibitor_categories_map, exhibitor_category_counts = get_exhibitor_category_info(exhibitors_df)

In [7]:
exhibitor_categories_map

{10846: {'agencies',
  'agency',
  'distributor',
  'gastro',
  'gastro tourism',
  'helicopter',
  'helicopter services',
  'machinery',
  'manufacturer',
  'medical',
  'medical products machinery manufacturer distributor',
  'national',
  'national park',
  'park',
  'payment',
  'payment service provider',
  'product',
  'products',
  'provider',
  'service',
  'services',
  'tourism',
  'travel',
  'travel agencies'},
 13033: {'banking',
  'catering',
  'companies',
  'company',
  'equipment',
  'equipment restaurants catering',
  'hostel',
  'hostel motel',
  'insurance',
  'insurance companies banking',
  'motel',
  'radio',
  'radio tv',
  'restaurant',
  'restaurants',
  'tv'},
 15642: {'aggregator',
  'apartment',
  'apartments',
  'apartments residential hotel',
  'chain',
  'engine',
  'event',
  'event organizer',
  'glampings',
  'home',
  'homes',
  'hotel',
  'hotel hotel chain inn',
  'inn',
  'mobile',
  'mobile homes glampings',
  'organizer',
  'residential',
  'sea

In [13]:
exhibitor_category_counts

{10846: 6,
 13033: 4,
 15642: 4,
 15881: 7,
 17729: 7,
 18004: 1,
 18960: 4,
 21865: 9,
 27827: 2,
 30134: 2,
 31331: 3,
 32278: 3,
 44942: 4,
 46350: 1,
 48028: 8,
 51839: 6,
 54579: 5,
 55311: 9,
 56923: 2,
 68142: 8,
 72153: 10,
 74870: 7,
 80700: 1,
 90556: 2,
 92462: 6,
 92491: 2,
 92492: 3,
 92493: 3,
 92494: 3,
 92495: 5,
 92496: 3,
 92497: 3,
 97818: 2,
 97920: 6,
 98889: 5}

In [ ]:
# Recommendation parameters
PENALTY_ALPHA = 0.3 # Penalty strength (0=no penalty, higher=stronger)
PENALTY_THRESHOLD = 6 # Number of categories before penalty increases significantly

#Currently considering only these questions since these questions are relevant to exhibitors and their categories.
RELEVANT_QUESTIONS = [
    "Please indicate your company's main area of business",
    "Which of the following best describes your job function?"
]

def recommend_exhibitors(
    visitor_email,
    processed_visitors_df,
    exhibitor_categories_map, # Pre-processed map {exhibitorid: {categories}}
    exhibitor_category_counts, # Pre-processed map {exhibitorid: count}
    exhibitor_names_map, # Map {exhibitorid: name}
    penalty_alpha=PENALTY_ALPHA,
    penalty_threshold=PENALTY_THRESHOLD
):
    """
    Recommends top N exhibitors for a given visitor email.

    Args:
        visitor_email (str): The email address of the visitor.
        processed_visitors_df (pd.DataFrame): DataFrame with processed visitor answers.
        exhibitor_categories_map (dict): Map of exhibitor IDs to their set of cleaned category names.
        exhibitor_category_counts (dict): Map of exhibitor IDs to their total unique category count.
        exhibitor_names_map (dict): Map of exhibitor IDs to their names.
        top_n (int): Number of recommendations to return.
        penalty_alpha (float): Penalty strength parameter.
        penalty_threshold (int): Threshold for penalty calculation.

    Returns:
        pd.DataFrame: Top N recommended exhibitors with scores and matching details.
                      Returns empty DataFrame if visitor not found or no matches.
    """
    # 1. Get visitor interests
    visitor_interests = get_visitor_interests(visitor_email, processed_visitors_df,RELEVANT_QUESTIONS)
    print(f"\nVisitor Interests for {visitor_email}: {visitor_interests}")

    scores = []

    # 2. Iterate through all exhibitors
    print(f"Calculating scores against {len(exhibitor_categories_map)} exhibitors...")
    for exhibitor_id, exhibitor_cats_set in tqdm(exhibitor_categories_map.items(), desc="Matching Exhibitors"):

        total_categories = exhibitor_category_counts[exhibitor_id]

        # 3. Calculate score
        score, num_matches, matched_categories = calculate_match_score(
            visitor_interests,
            exhibitor_cats_set,
            total_categories,
            penalty_alpha,
            penalty_threshold
        )

        # 4. Store results if score > 0
        if score > 0:
            scores.append({
                'exhibitorid': exhibitor_id,
                'exhibitorName': exhibitor_names_map.get(exhibitor_id, "Unknown Name"),
                'score': score,
                'numMatches': num_matches,
                'matchedCategories': ', '.join(sorted(list(matched_categories))) # Convert set to string
            })

    # 5. Sort and return top N
    if not scores:
        print("No matching exhibitors found for this visitor's interests.")
        return pd.DataFrame(columns=['exhibitorid', 'exhibitorName', 'score', 'numMatches', 'matchedCategories'])

    recommendations_df = pd.DataFrame(scores)
    recommendations_df = recommendations_df.sort_values(by='score', ascending=False)

    return recommendations_df

In [9]:

# Example usage
test_visitor_email = 'emilija+100_L8gA@bss.mk' 
print(f"\n--- Generating recommendations for: {test_visitor_email} ---")

recommendations = recommend_exhibitors(
    test_visitor_email,
    visitors_df,
    exhibitor_categories_map,
    exhibitor_category_counts,
    exhibitor_names
)

recommendations


--- Generating recommendations for: emilija+100_L8gA@bss.mk ---

Visitor Interests for emilija+100_L8gA@bss.mk: {'travel agent', 'media', 'agent', 'medium', 'travel'}
Calculating scores against 35 exhibitors...


Matching Exhibitors: 100%|██████████| 35/35 [00:00<00:00, 35144.99it/s]


,exhibitorid,exhibitorName,score,numMatches,matchedCategories
11,74870,Exotic Tours Journeys,2.307692,3,"media, medium, travel"
2,18960,Elite Travel Services,2.000000,2,"media, medium"
13,92492,Asia Tourism,2.000000,2,"media, medium"
8,55311,Sunny Adventures Journeys,1.578947,3,"media, medium, travel"
10,72153,Elite Adventures Holidays,1.363636,3,"media, medium, travel"
0,10846,Royal Adventures Adventures,1.000000,1,travel
1,15642,Dream Travel Services,1.000000,1,travel
4,27827,Exotic Tours Holidays,1.000000,1,travel
14,92494,Exotic Europe Travels,1.000000,1,travel
7,44942,Royal Holidays Adventures,1.000000,1,travel


In [10]:
# Top 7 Recommendations
recommendations.head(7)

,exhibitorid,exhibitorName,score,numMatches,matchedCategories
11,74870,Exotic Tours Journeys,2.307692,3,"media, medium, travel"
2,18960,Elite Travel Services,2.000000,2,"media, medium"
13,92492,Asia Tourism,2.000000,2,"media, medium"
8,55311,Sunny Adventures Journeys,1.578947,3,"media, medium, travel"
10,72153,Elite Adventures Holidays,1.363636,3,"media, medium, travel"
0,10846,Royal Adventures Adventures,1.000000,1,travel
1,15642,Dream Travel Services,1.000000,1,travel
